# Gradient Descent Algorithms

This notebook is an exploraton of gradient descent algorithms. We explore two dimensions of variation here. The first pertains to the calculation of the magnitude of the descent (the learning rate, $\alpha$) along the gradient at each iteration of the algorithm. The second pertains to variations in the number of training samples utilized in the calculation of the gradient $\triangledown$ of the cost $J$ with respect to $\theta_j$.

### Basic Gradient Descent Algorithm

Gradient descent is essentially a method for calculating parameters that when applied to each of the observations (also called dependent variables or features) in a data set $X$ results in the lowest total error, or cost.


$$f(X,\theta)=H$$


where $H$ is the estimated value of the output (also called dependent variables or labels) for each set of inputs and

$$J(\theta) = \sum_{i=1}^m\mid{h_i(\theta,x_i) - y_i}\mid$$

where $J$ represents the total cost. Different methods of computing the cost are used depending on the application, but the formula above expresses the essence of it, which is that we are trying to predict the actual outcomes as closely as possible.

In order to minimze $J$ with respect to $\theta$ we start with an abitrary value of $\theta$ and then calculate the direction for each our input variables that would result in the fastest decrease in cost (gradient). Then we calculate a new theta based on the gradient and a model input for the magnitude of the change (the learning rate or $\alpha$). Then we repeat that process until $\theta$ and $J$ stop changing and that should result in a value of $\theta$ that minimizes $J$.


$$\theta_j := \theta_j - \alpha\triangledown\theta_jJ(\theta)$$

or

$$\theta_j := \theta_j - \alpha\frac\partial{\partial\theta_j}J(\theta)$$

#### Normal Equation

$\theta$ can also be calculated directly using the equation below.

$$\theta = (X^TX)^{-1}X^Ty$$

Complexity for the normal equation, $O(n^3)$, is greater than it is it for the basic gradient descent algorithm, $O(n^2)$, but has acceptable performance up to approximately $n=10^5$ features.

### Methods for Determining Learning Rate

We explore three methods for determining the learning rate.

1. Constant
2. Adaptive Gradient Evaluation
3. Adapative Momentum Estimation

In order to compare the performance of the algorithms and ensure that they are functioning correctly, we will use a known function as our cost function. This allows us to compare the value of $\theta$ produced by each algorithm and its associated $J$ with values we can calculate directly from our known function.  Using a known function with two dimensional inputs allows us to plot as a surface all possible values of $J$ for a given range of values for $\theta$ and then $J_\theta$ for each iteration of the algorithm.  We will use the [Stablinsky-Tang function](https://en.wikipedia.org/wiki/Test_functions_for_optimization), which results in a non-convex cost surface suitable for testing with straightforward gradient and cost computations.

Our gradient descent algorithm has been generalized to accept the following arguments:

|Argument        |Definition                                                              |
|----------------|------------------------------------------------------------------------|
|`theta_hist`    |Starting value of $\theta_0$ and calculated value of $J_{\theta_0}$     |
|`alpha`         |Learning rate, $\alpha$                                                 |
|`get_gradient`  |Function that returns gradient, $f(\theta_j)=\triangledown_{\theta_j}$  |
|`get_cost`      |Function that returns cost, $f(\theta_j)=J_{\theta_j}$                  |
|`delta_min`     |Minimum change in cost to establish convergence                         |

Throughout the variations we consider, the core algorithm remains the same. Both changes to the calculation of the learning rate, $\alpha$, and later, the number of training samples used in calculating the gradient, are effected by modifying the function that gets passed in with respect to the `get_gradient` argument.

##### Cost Function

The Styblinski–Tang function with respect to $\theta$ is:

$$J(\theta) = \dfrac{\sum_{i=1}^n\theta_i^4-16\theta_i^2+5\theta_i}{n}$$

where $n$ is the number of dimensions in the data. For two dimensions, we can also express our cost function as:

$$J(\theta) = \dfrac{\theta_1^4-16\theta_1^2+5\theta_1+\theta_2^4-16\theta_2^2+5\theta_2}{2}$$

The global minimum of this function is $-78.33233$ at $\theta ~ (-2.9,-2.9)$

##### Gradient Function

$$\frac\partial{\partial\theta_n}J(\theta) = 2\theta_n^3-16\theta_n+2.5$$

##### Plots

The color scale of the surface corresponds to the z-axis value, which represents total cost, $J$, for all values of theta in the displayed range. The color scale of the points on the surface, which represent the total cost, $J_{\theta_j}$, as function of $\theta_j$ at each iteration of the model, corresponds to the iteration.

The plots are also interactive, so you can spin them around and zoom in and out for more detail.

### Constant Learning Rate

In [11]:
import numpy as np
import plotly.offline as py
py.init_notebook_mode()
import plotly.graph_objs as go

# Define gradient function
def grad_fun_st(theta):
    return np.apply_along_axis(lambda o: 2.5 - 16*o + 2*o**3, 0, theta)

# Define cost function
def cost_fun_st(theta):
    return np.append(theta, np.sum(5*theta - 16*theta**2 + theta**4) / theta.shape[0])

# Define gradient generator function for constant learning rate
# Allows use of alternative gradient calculations
def grad_const_gen(theta, grad_fun):
    while True:
        yield grad_fun(theta)
    
# Perform gradient descent algorithm
def gd(theta_hist, alpha, grad_gen, grad_fun, cost_fun, delta_min):

    # Initialize iteration variables
    delta = float("inf")
    i = 0
    
    # Run algorithm
    while delta > delta_min:
        # Get theta
        theta = theta_hist[i,:-1]
        
        # Calculate next next theta from alpha and gradient
        theta = theta - alpha * next(grad_gen(theta, grad_fun))
        
        # Calculate cost for convergence test and store with theta for plot
        try:
            theta_hist[i+1] = cost_fun(theta)
        except:
            print('{} minimum change not achieved in {} iterations.'.format(delta_min, theta_hist.shape[0]))
            break
        delta = abs(theta_hist[i-1,-1] - theta_hist[i,-1])
        i += 1
    
    # Trim zeros
    theta_hist = theta_hist[np.nonzero(theta_hist[:,-1])]
    return theta_hist

# Initialize model hyperparameters
delta_min = 10**-6
alpha = 0.01

# Initialize starting value of theta
theta0 = np.array([-0.2, -4.4])
theta_hist = np.zeros((1000, theta0.shape[0]+1))
theta_hist[0] = cost_fun_st(theta0)

# Run algorithm
theta_hist = gd(theta_hist, alpha, grad_const_gen, grad_fun_st, cost_fun_st, delta_min)

In [12]:
# Prepare plot
# Prepare surface
x = np.arange(-4.6, 4.6, 0.1)
y = np.arange(-4.6, 4.6, 0.1)
X, Y = np.meshgrid(x, y)
Z = 1/2.0 * (X**4 - 16*X**2 + 5*X + Y**4 - 16*Y**2 + 5*Y)

# Prepare surface contours
contour = dict(
    show = True,
    color = 'DodgerBlue', #'#0066FF',
    highlightcolor = 'DeepSkyBlue',
    highlightwidth = 1.5,
    width = 1
)

# Add surface to plot
surface = go.Surface(
    name = 'J surface',
    x = X,
    y = Y,
    z = Z,
    colorscale = 'Rainbow',
    showlegend = False,
    contours = dict(
        y = contour,
        x = contour,
        z = dict(
            show = False,
            color = contour['color'],
            highlightcolor = contour['highlightcolor'],
            highlightwidth = contour['highlightwidth'],
            width = contour['width']
        )
    )
)

# Add theta_hist to plot - set up as a function for future plots
def spec_theta_hist_trace(theta_hist):
    theta_hist_trace = go.Scatter3d(
        name = 'theta_hist',
        x = theta_hist[:,0],
        y = theta_hist[:,1],
        z = theta_hist[:,2],
        mode = 'markers',
        showlegend = False,
        marker = dict(
            color = np.arange(theta_hist.shape[0]),
            colorscale = 'Blackbody',
            showscale = False,
            size = "5"
        )
    )
    return theta_hist_trace

# Specify layout options
layout = go.Layout(
    title='Constant Alpha',
    autosize=False,
    width=700,
    height=700,
    scene=dict(
        xaxis=dict(
            title = 'theta1',
            ticks = "outside",
            dtick = 0.25,
            showticklabels = False
        ),
        yaxis=dict(
            title = 'theta2',
            ticks = "",
            dtick = 0.25,
            showticklabels = False
        ),
        zaxis=dict(
            title = 'J',
        ),
        camera=dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=0.25, y=1.25, z=1.15)
        )
    )
)

# Execute plot
fig = go.Figure(data=[surface, spec_theta_hist_trace(theta_hist)], layout=layout)
py.iplot(fig, filename='constant_alpha_gradient_descent')

You can see from this graph that the core gradient descent algorithm is functioning as expected. With a starting value of $\theta=(-0.3,4.6)$, it produces a final $\theta$ that minimizes total cost, $J$. If you spin the graph around (it's actually easiest to see looking at the bottom of the surface), you can see the steps in $\theta$ the algorithm produced. Visually they appear to be in the direction of the greatest reduction in cost. The steps are also bigger where the gradient is steeper and smaller where it is flatter. The rate of change looks like it changed four times on the way to the global minimum.

This algorithm's ability to solve for the global minimum of $J$ depends on $\alpha$ and the starting value of $\theta_0$. You can try different values of $\alpha$ and $\theta_0$ to see how they affect the algorithm's ability to successfully solve for $\theta$ such that the global minimum cost, $J_{min}$, is achieved. Even in the scenarios where the algorithm fails to minimize cost, it appears to be functioning correctly.

## Adaptive Gradient Algorithm (Adagrad)

[Duchi, J., Hazan, E., and Singer, Y. Adaptive Subgradient Methods for Online Learning and Stochastic Optimization](http://stanford.edu/~jduchi/projects/DuchiHaSi10_colt.pdf)

In [13]:
# Define gradient generator
def grad_gen_adagrad(theta, grad_fun):
    # Initialize gradient history and hyperparameter epsilon
    grad_hist = 0
    epsilon = 10**-8

    # Generate gradient
    while True: 
        # Get gradient to adapt from gradient function
        gradient = grad_fun(theta)
        
        # Perform gradient adaptation
        grad_hist += np.square(gradient)
        yield gradient / (epsilon + np.sqrt(grad_hist))

# Initialize hyperparamters
alpha = 0.1
delta_min = 10**-3

# Initialize starting value of theta
theta0 = np.array([-0.2, -4.4])
theta_hist = np.zeros((10000, theta0.shape[0]+1))
theta_hist[0] = cost_fun_st(theta0)

# Run algorithm
theta_hist = gd(theta_hist, alpha, grad_gen_adagrad, grad_fun_st, cost_fun_st, delta_min)

0.001 minimum change not achieved in 10000 iterations.


In [14]:
# Execute plot
layout.title = 'Adaptive Gradient Descent'
fig = go.Figure(data=[surface, spec_theta_hist_trace(theta_hist)], layout=layout)
py.iplot(fig, filename='adagrad_gradient_descent')

## Adaptive Moment Estimation (Adam)

[Kingma, D. P., & Ba, J. L. (2015). Adam: A Method for Stochastic Optimization. International Conference on Learning Representations.](https://arxiv.org/pdf/1412.6980v8.pdf)

In [15]:
# Define gradient generator
def grad_gen_adam(theta, grad_fun):
    # Initialize moment variables and hyperparameters
    moment1 = 0
    moment2 = 0
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 10**-8
    i = 1
    
    # Generate gradient
    while True: 
        # Get gradient to adapt from gradient function
        gradient = grad_fun(theta)
        
        # Update moment estimates
        moment1 = beta1 * moment1 + (1 - beta1) * gradient
        moment2 = beta2 * moment2 + (1 - beta2) * np.square(gradient)
        moment1_hat = moment1 / (1 - beta1**i)
        moment2_hat = moment2 / (1 - beta2**i)
        i += 1
        
        # Yield adapted gradient
        yield theta - alpha * moment1_hat / (epsilon + np.sqrt(moment2_hat))

# Initialize hyperparamters
alpha = 0.01
delta_min = 10**-6

# Initialize starting value of theta
theta0 = np.array([-2.0, -4.4])
theta_hist = np.zeros((1000, theta0.shape[0]+1))
theta_hist[0] = cost_fun_st(theta0)

# Run algorithm
theta_hist = gd(theta_hist, alpha, grad_gen_adam, grad_fun_st, cost_fun_st, delta_min)

1e-06 minimum change not achieved in 1000 iterations.


In [16]:
# Execute plot
layout.title = 'Adaptive Moment Estimation'
fig = go.Figure(data=[surface, spec_theta_hist_trace(theta_hist)], layout=layout)
py.iplot(fig, filename='adam_gradient_descent')

**!!!Adam - to be fixed - appears not to be working as it always converges to $\theta = (0,0)$.**

## Batching
Here we add in the capablility to calculate the gradient from batches of training samples.

In [20]:
def batch_gen(data, sizeB):
    i = 0
    np.random.shuffle(data)
    while i < int(data.shape[0] / sizeB):
        indexB = slice(i*sizeB, min((i+1)*sizeB, data.shape[0]), 1)
        yield (data[indexB,:-1], data[indexB,-1])
        i += 1

## Linear Regression

In [18]:
# batch is tuple of (X values, y values)
def grad_fun_linear(batch, theta):
    X, y = batch
    return X.T.dot(X.dot(theta) - y) / X.shape[0]

def cost_fun_linear(data, theta):
    X = data[:,:-1]
    y = data[:,-1]
    return np.sum(np.square(X.dot(theta) - y)) / (2 * data.shape[0])

## Logistic Regression

With logistic regression, we have a depedent variable that is classified, meaning it can only assume a finite number of values. In order to account for this we modify our cost function such that our predictions will always range between $0$ and $1$.

$$\begin{align*}& h_\theta (x) =  g ( \theta^T x ) \newline \newline& z = \theta^T x \newline& g(z) = \dfrac{1}{1 + e^{-z}}\end{align*}$$

In [19]:
def grad_fun_logistic(batch, theta):
    X, y = batch
    sigmoid = 1 / (1 + np.exp(-X.dot(theta)))
    return (sigmoid - y).T.dot(X) / X.shape[0]

def cost_fun_logistic(data, theta):
    X = data[:,:-1]
    y = data[:,-1]
    sigmoid = 1 / (1 + np.exp(-X.dot(theta)))
    return np.sum(-y.dot(log(simoid)) - (1 - y).dot(log(1 - sigmoid))) / data.shape[0]